In [1]:
#IMPORTO LIBRERIE
import pandas as pd

In [2]:
labeled_df = pd.read_csv('labeled_df.csv')
labeled_df.shape

(90960, 16)

In [3]:
labeled_df = labeled_df.drop(columns='Unnamed: 0')
labeled_df.head()

,customerId,timestamp,country,amount,businessName,userAgent_isPc,userAgent_browserFamily,userAgent_osFamily,IBAN_countryCode,IBAN_bankCode,SIA,CAP,SAE,RAE,companyType
0,93000801,2021-09-15 16:13:35.147,6,150.00,c7543b6f04850eae288aa52783b33363,1,4,5,17,699,2013,144,19,113,5
1,93000801,2021-08-24 10:34:24.487,5,3200.00,c7543b6f04850eae288aa52783b33363,1,4,5,17,699,2013,144,19,113,5
2,93000801,2021-09-14 16:21:43.33,6,8350.00,c7543b6f04850eae288aa52783b33363,1,4,5,17,699,2013,144,19,113,5
3,93001001,2021-06-30 09:07:16.733,6,500.00,3f241dfcc38064261fec0f303c2784e5,1,0,5,17,123,1781,90,8,99,10
4,93001001,2021-07-12 16:36:02.857,6,1895.68,a5d1eaffa8e325d370665f43624339a1,1,0,5,17,285,1781,90,8,99,10


In [ ]:
from pyod.models.ecod import ECOD
X=labeled_df.copy()
X=X.drop(columns=['businessName','customerId','timestamp'])
clf = ECOD(contamination=0.001)
clf.fit(X)
clf.get_params()

In [ ]:
import pickle
encoder_dict=pickle.load(open('enc_dict.pkl','rb'))
columns_to_label = ['country', 'userAgent_browserFamily', 'userAgent_osFamily', 'IBAN_countryCode', 'IBAN_bankCode','CAP', 'SIA', 'SAE', 'RAE','companyType']

In [ ]:
def inverse_and_scores(labeled_df,enc_dict,columns,model):
    df_copy=labeled_df.copy()
    labels=model.labels_
    scores = model.decision_scores_

    for clmn in columns:
        df_copy[clmn]=enc_dict[clmn].inverse_transform(df_copy[clmn])

    df_copy['scores'] = scores
    df_copy['is_anomaly'] = labels

    return df_copy

df_scores=inverse_and_scores(labeled_df,encoder_dict,columns_to_label,clf)

df_scores.head()

In [ ]:
df_scores[df_scores['is_anomaly']==1].sort_values('scores',ascending=False)

In [ ]:
df_scores.to_csv('ECOD_scores.csv')

In [ ]:
ECOD_labeled_df = X.copy()
ECOD_labeled_df['label']=clf.labels_
ECOD_labeled_df.head()

In [ ]:
ECOD_labeled_df.to_csv('ECOD_labeled_df.csv')

In [ ]:
test=pd.read_csv('misc_test.csv',decimal=',')
test.head()

In [ ]:
columns_to_label = ['country', 'userAgent_browserFamily', 'userAgent_osFamily', 'IBAN_countryCode', 'IBAN_bankCode','CAP', 'SIA', 'SAE', 'RAE','companyType']

for clmn in columns_to_label:
    test[clmn]=test[clmn].apply(str)
    test[clmn] = encoder_dict[clmn].transform(test[clmn])

test.head()

In [ ]:
clf.predict(test)

In [6]:
test=pd.read_csv('misc_test.csv',decimal=',')
test.head()

,country,amount,userAgent_isPc,userAgent_browserFamily,userAgent_osFamily,IBAN_countryCode,IBAN_bankCode,SIA,CAP,SAE,RAE,companyType
0,Austria,264986.74,1,Chrome,Windows,AT,34000,BUV4U,39031,430,161,SRL
1,Italy,111345.77,1,Chrome,Windows,DE,34270024,B5SDZ,39100,615,11,SS
2,Italy,1659.00,1,Firefox,Windows,IT,3069,BQ6RB,31010,430,649,SRL
3,Italy,200.00,1,Safari,Mac OS X,IT,8794,CM5CR,30174,492,614,SAS
4,Italy,1000.00,1,Chrome,Windows,IT,8327,CEFGV,31039,430,505,SRL


In [7]:
columns_to_label = ['country', 'userAgent_browserFamily', 'userAgent_osFamily', 'IBAN_countryCode', 'IBAN_bankCode','CAP', 'SIA', 'SAE', 'RAE','companyType']

for clmn in columns_to_label:
    test[clmn]=test[clmn].apply(str)
    test[clmn] = encoder_dict[clmn].transform(test[clmn])

test.head()

,country,amount,userAgent_isPc,userAgent_browserFamily,userAgent_osFamily,IBAN_countryCode,IBAN_bankCode,SIA,CAP,SAE,RAE,companyType
0,0,264986.74,1,0,5,0,364,1482,302,8,4,10
1,6,111345.77,1,0,5,6,376,965,327,25,1,11
2,6,1659.00,1,4,5,17,285,1346,62,8,121,10
3,6,200.00,1,11,3,17,1217,2045,58,19,106,5
4,6,1000.00,1,0,5,17,1088,1781,87,8,99,10


In [8]:
clf.predict(test)

array([0, 0, 0, 0, 0, 0, 1, 1, 0])